In [31]:
# Import Libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

In [32]:
df=pd.read_csv("../Dataset/bike_data.csv")

In [33]:
df.head()

,Date,Season,Hour,Holiday,Day of the Week,Working Day,Weather Type,Temperature F,Temperature Feels F,Humidity,Wind Speed,Casual Users,Registered Users,Total Users
0,1/1/2011,4,0,0,6,0,1,36.6,37.4,81,0,3,13,16
1,1/1/2011,4,1,0,6,0,1,34.9,35.6,80,0,8,32,40
2,1/1/2011,4,2,0,6,0,1,34.9,35.6,80,0,5,27,32
3,1/1/2011,4,3,0,6,0,1,36.6,37.4,75,0,3,10,13
4,1/1/2011,4,4,0,6,0,1,36.6,37.4,75,0,0,1,1


In [34]:
df.isnull().sum()

Date                   0
Season                 0
Hour                   0
Holiday                0
Day of the Week        0
Working Day            0
Weather Type           0
Temperature F          0
Temperature Feels F    0
Humidity               0
Wind Speed             0
Casual Users           0
Registered Users       0
Total Users            0
dtype: int64

In [35]:
df = df.drop(['Date'], axis=1)

In [36]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17379 entries, 0 to 17378
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Season               17379 non-null  int64  
 1   Hour                 17379 non-null  int64  
 2   Holiday              17379 non-null  int64  
 3   Day of the Week      17379 non-null  int64  
 4   Working Day          17379 non-null  int64  
 5   Weather Type         17379 non-null  int64  
 6   Temperature F        17379 non-null  float64
 7   Temperature Feels F  17379 non-null  float64
 8   Humidity             17379 non-null  int64  
 9   Wind Speed           17379 non-null  int64  
 10  Casual Users         17379 non-null  int64  
 11  Registered Users     17379 non-null  int64  
 12  Total Users          17379 non-null  int64  
dtypes: float64(2), int64(11)
memory usage: 1.7 MB


In [37]:
df.describe()

,Season,Hour,Holiday,Day of the Week,Working Day,Weather Type,Temperature F,Temperature Feels F,Humidity,Wind Speed,Casual Users,Registered Users,Total Users
count,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000
mean,2.485241,11.546752,0.028770,3.003683,0.682721,1.425283,58.777507,59.722009,62.722884,12.736233,35.676218,153.786869,189.463088
std,1.115985,6.914405,0.167165,2.005771,0.465431,0.639357,16.620757,20.415345,19.292983,8.196891,49.305030,151.357286,181.387599
min,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,17.600000,3.200000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,1.000000,6.000000,0.000000,1.000000,0.000000,1.000000,45.200000,42.800000,48.000000,7.000000,4.000000,34.000000,40.000000
50%,2.000000,12.000000,0.000000,3.000000,1.000000,1.000000,59.000000,60.800000,63.000000,13.000000,17.000000,115.000000,142.000000
75%,3.000000,18.000000,0.000000,5.000000,1.000000,2.000000,72.800000,77.000000,78.000000,17.000000,48.000000,220.000000,281.000000
max,4.000000,23.000000,1.000000,6.000000,1.000000,4.000000,102.200000,122.000000,100.000000,57.000000,367.000000,886.000000,977.000000


In [38]:
df

,Season,Hour,Holiday,Day of the Week,Working Day,Weather Type,Temperature F,Temperature Feels F,Humidity,Wind Speed,Casual Users,Registered Users,Total Users
0,4,0,0,6,0,1,36.6,37.4,81,0,3,13,16
1,4,1,0,6,0,1,34.9,35.6,80,0,8,32,40
2,4,2,0,6,0,1,34.9,35.6,80,0,5,27,32
3,4,3,0,6,0,1,36.6,37.4,75,0,3,10,13
4,4,4,0,6,0,1,36.6,37.4,75,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
17374,4,19,0,1,1,2,38.3,33.8,60,11,11,108,119
17375,4,20,0,1,1,2,38.3,33.8,60,11,8,81,89
17376,4,21,0,1,1,1,38.3,33.8,60,11,7,83,90
17377,4,22,0,1,1,1,38.3,35.6,56,9,13,48,61


In [54]:
x=df["Registered Users"]
y=df["Total Users"]
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2)

#reshape the data
x_train = x_train.to_numpy()
x_test = x_test.to_numpy()
y_train = y_train.to_numpy()
y_test = y_test.to_numpy()

x_train = np.reshape(x_train, (-1,1))
x_test = np.reshape(x_test, (-1,1))
y_train = np.reshape(y_train, (-1,1))
y_test = np.reshape(y_test, (-1,1))

print('Train - Predictors shape', x_train.shape)
print('Test - Predictors shape', x_test.shape)
print('Train - Target shape', y_train.shape)
print('Test - Target shape', y_test.shape)

model = LinearRegression()
model.fit(x_train,y_train)
model.score(x_test,y_test)

Train - Predictors shape (13903, 1)
Test - Predictors shape (3476, 1)
Train - Target shape (13903, 1)
Test - Target shape (3476, 1)


0.9414915522030536

In [55]:
#finding MSE value with test data
from sklearn.metrics import mean_squared_error
y_preds=model.predict(x_test)
mse=mean_squared_error(y_test,y_preds)
mse

2028.631707945845

In [56]:
# finding MSE value with train data

# from sklearn.metrics import mean_squared_error
# y_preds=model.predict(x_train)
# mse=mean_squared_error(y_train,y_preds)
# mse

In [57]:
r2_score(y_test,y_preds)

0.9414915522030536